In [23]:
import os
import sys
import pandas as pd
import yaml 
from matplotlib import pyplot as plt
from matplotlib import ticker as mticker
import statsmodels.api as sm
import numpy as np
from itertools import product
import subprocess

with open("../../config.yaml.local", "r") as f:
    LOCAL_CONFIG = yaml.safe_load(f)
#with open("../../config.yaml", "r") as f:
#    CONFIG = yaml.safe_load(f)
sys.path.append("../python")

import globals
import data_tools as dt
import writing_tools as wt

LOCAL_PATH = LOCAL_CONFIG["LOCAL_PATH"]
RAW_DATA_PATH = LOCAL_CONFIG["RAW_DATA_PATH"]
DATA_PATH = LOCAL_CONFIG["DATA_PATH"]
R_PATH = LOCAL_CONFIG["R_PATH"]

RUN_R_SCRIPTS = False

RESULTS = {}

In [24]:
# Output data for post quality regressions
df1 = dt.get_post_quality_analysis_data()
df1.to_csv(os.path.join(DATA_PATH, 'post_quality_analysis_data.csv'), index=False)

c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:55: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  return x.dt.to_period('W-SAT').dt.start_time
c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:55: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  return x.dt.to_period('W-SAT').dt.start_time


In [25]:
# Output data for post quantity regressions
df2 = dt.get_post_quantity_analysis_data()
df2.to_csv(os.path.join(DATA_PATH, 'post_quantity_analysis_data.csv'), index=False)

c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:55: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  return x.dt.to_period('W-SAT').dt.start_time
c:\Users\edwar\projects\sn-research\src\notebooks\../python\data_tools.py:55: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  return x.dt.to_period('W-SAT').dt.start_time


In [26]:
res = subprocess.run([R_PATH, LOCAL_PATH + "/src/R/post-cost-regressions.R"], check=True, capture_output=True, text=True)
#print(res.stdout)

In [27]:
quantity_reg_coefs = pd.read_csv(
    os.path.join(DATA_PATH, 'posts_cost_reg.csv'),
    index_col = 0
)
coef = quantity_reg_coefs.loc['log_cost', 'coef(rquan3)']
elast = fr"{coef:.3f}"
double_effect = np.exp(coef*np.log(2)) - 1
double_effect_pct = fr"{np.abs(double_effect)*100:.1f}"
RESULTS['PostsCostElasticity'] = elast
RESULTS['PostsCostDoublingEffect'] = double_effect_pct
wt.update_results(RESULTS)


{'PostsCostElasticity': '-0.244',
 'PostsCostDoublingEffect': '15.6',
 'Zaps48CostElasticity': '0.187',
 'Zaps48CostDoublingEffect': '13.8'}

In [28]:
quality_reg_coefs = pd.read_csv(
    os.path.join(DATA_PATH, 'sats48_cost_reg.csv'),
    index_col = 0
)
coef = quality_reg_coefs.loc['log_cost', 'coef(rzaps3)']
elast = fr"{coef:.3f}"
double_effect = np.exp(coef*np.log(2)) - 1
double_effect_pct = fr"{np.abs(double_effect)*100:.1f}"
RESULTS['Zaps48CostElasticity'] = elast
RESULTS['Zaps48CostDoublingEffect'] = double_effect_pct
wt.update_results(RESULTS)


{'PostsCostElasticity': '-0.244',
 'PostsCostDoublingEffect': '15.6',
 'Zaps48CostElasticity': '0.187',
 'Zaps48CostDoublingEffect': '13.8'}